[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

In [4]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [4]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard; the dashboard should contain the logic to wrangle such JSON files into DataFrames. These should be used to display initial graphs. The wrangling code will also come in handy for the next step. 

In this template, we just generate some random data (the code below will be hidden when the dashboard is rendered by Voila).

In [5]:
with open ("deaths_adm.json", "rt") as INFILE:
    data = json.load(INFILE)
    
datalist=data["data"]

dates = [dictionary['date'] for dictionary in datalist]

dates.sort()

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")


startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

index=pd.date_range(startdate, enddate, freq='D')

deaths_adm_df=pd.DataFrame(index=index, columns=['Admissions','Deaths'])


for entry in datalist:
    date=parse_date(entry['date'])
    for column in ['Admissions','Deaths']:
        if pd.isna(deaths_adm_df.loc[date, column]):
            value = float(entry[column]) if entry[column] != None else 0.0
            
            deaths_adm_df.loc[date, column] = value
            




In [ ]:
with open ("testsVScases.json", "rt") as INFILE:
    data = json.load(INFILE)
    
datalist=data["data"]

dates = [dictionary['date'] for dictionary in datalist]

dates.sort()

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")


startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

index=pd.date_range(startdate, enddate, freq='D')

testsVcases_df=pd.DataFrame(index=index, columns=['One','Two','Cases'])


for entry in datalist:
    date=parse_date(entry['date'])
    for column in ['One','Two','Cases']:
        if pd.isna(deaths_adm_df.loc[date, column]):
            value = float(entry[column]) if entry[column] != None else 0.0
            
            testsVcases_df.loc[date, column] = value

df=pd.DataFrame(index=range(0,100), columns=['One', 'Two'])

def create_randomdata():
    # generate two random walks.
    one=two=0.0
    for i in range(0,100):
        df.loc[i,'One']=one
        df.loc[i,'Two']=two
        one+=np.random.randn()
        two+=2*np.random.randn()
        
# the code above should really be reading the json files and preparing the dataframes you want to plot.
# putting the wrangling code into a function allows you to call it again after refreshing the data through the 
# API. You can obviously call the function directly when the dashboard starts, by just including the call in 
# the cell as below:
create_randomdata()

## Download current data

Give your users an option to refresh the dataset - a "refresh" button that calls the code to access the PHE API will do. You can then either
* overwrite the ```.json``` files and launch code that re-reads and wrangles them or better
* skip the files altogether and call the wrangling code directly (faster).

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not deleted and nothing crashes if for any reason the server cannot be reached or data is not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [9]:
# Place your API access code in a function that will be called by the button.
# If you have time, include some error handling around the API access call.
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def access_api(button):
    filters = ["areaType=overview"]

    structure={"date":"date", "Admissions": "newPillarOneTestsByPublishDate", 
           "Deaths": "newDeaths28DaysByPublishDate"}

    api = Cov19API(filters=filters, structure=structure)

    deaths_adm = api.get_json()
    
    with open("deaths_adm.json", "wt") as OUTF:
        json.dump(deaths_adm, OUTF)
    
    with open ("deaths_adm.json", "rt") as INFILE:
        data = json.load(INFILE)
    
    datalist=data["data"]

    dates = [dictionary['date'] for dictionary in datalist]

    dates.sort()
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")


    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')

    deaths_adm_df=pd.DataFrame(index=index, columns=['Admissions','Deaths'])


    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['Admissions','Deaths']:
            if pd.isna(deaths_adm_df.loc[date, column]):
                value = float(entry[column]) if entry[column] != None else 0.0
            
                deaths_adm_df.loc[date, column] = value






    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see below
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use
    # "unlink" or "times" and change the button text to "Unavailable" in case of
    # failure.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='refresh data', # you may want to change this...
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="click here to update the graph with the latest data from PHE",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your api access function with the button
apibutton.on_click(access_api) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='success', description='refresh data', icon='exclamation-triangle', style=ButtonStyle(), t…

NameError: name 'refresh_graph' is not defined

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [6]:
series=wdg.SelectMultiple(
    options=['Admissions', 'Deaths'],
    value=['Admissions','Deaths'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    value='linear' , # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def deaths_adm(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        deaths_adm_df[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


graph=wdg.interactive_output(deaths_adm, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work!